In [6]:
import hashlib
import numpy as np
from numpy import array

completed_lines_hash = set()

completed_lines_hash = set()

#Save processed data to SMILES.txt
new = open("smiles.txt", "w")

#Read in data file line by line
for line in open("data.txt", "r"):
  
    #Ensure all smiles in original data file are unique
    hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()
  
    if hashValue not in completed_lines_hash:
        completed_lines_hash.add(hashValue)
        
        #Ensure all SMILES are between 35 and 75 characters in length
        if 33 < len(line) < 75:
            #Add start token
            line = line.rjust(len(line)+1, "G")
            
            #Remove "\n" characters, add end token
            line = line.rstrip('\n') + "E"

            #Copy over SMILES satisfying requirements
            new.write(line)
    
#Close files
new.close()

In [7]:
#Read in processed data file
data = open("smiles.txt", "r").read()

#Create a list of the unique characters in the dataset
chars = list(set(data))

#Get size (in characters) of dataset
data_size = len(data) 

#Get number of unique characters in dataset
vocab_size = len(chars)

#Print dataset properties
print("Vocab size: " + str(vocab_size))
print("Data size: " + str(data_size))
print("Characters in data: " + str(chars))

Vocab size: 53
Data size: 25246858
Characters in data: ['3', 'H', 'g', '5', 'l', '6', 'L', '/', 'F', 'c', '.', 'b', 'p', 'R', '[', '9', 'S', 'N', 'e', 'Z', '@', '7', '1', 'C', 'O', '=', '(', '-', 'i', 'o', 'V', '4', 'G', 'n', ']', 'K', 't', '#', 'B', 'P', '8', '2', ')', '+', '\\', 's', 'a', 'T', 'A', 'r', 'M', 'I', 'E']


In [8]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Create array from characters in the dataset
values = array(chars)
print("Array of unique characters:")
print(values)

#Create unique, numerical labels for each character between 0 and n-1, where n is the number of unique characters
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print("Array of labels for each character:")
print(integer_encoded)

#Encode characters into a one-hot encoding, resulting in an array of size [num unique chars, num unique chars]
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print("Array of one-hot encoded characters:")
print(onehot_encoded)
print("Size of array of one-hot encoded characters: " + str(onehot_encoded.shape))

Array of unique characters:
['3' 'H' 'g' '5' 'l' '6' 'L' '/' 'F' 'c' '.' 'b' 'p' 'R' '[' '9' 'S' 'N'
 'e' 'Z' '@' '7' '1' 'C' 'O' '=' '(' '-' 'i' 'o' 'V' '4' 'G' 'n' ']' 'K'
 't' '#' 'B' 'P' '8' '2' ')' '+' '\\' 's' 'a' 'T' 'A' 'r' 'M' 'I' 'E']
Array of labels for each character:
[ 9 24 44 11 46 12 27  6 22 42  5 41 49 32 37 15 33 29 43 36 17 13  7 20
 30 16  1  4 45 48 35 10 23 47 39 26 52  0 19 31 14  8  2  3 38 51 40 34
 18 50 28 25 21]
Array of one-hot encoded characters:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Size of array of one-hot encoded characters: (53, 53)


C:\Users\jake\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:
#Read in processed data file
data = open("smiles.txt", "r").read()
#Create a list of the dataset
datalist = list(data)
#Create an array of the dataset
dataarray = array(datalist)
#Fit one-hot encoding to dataarray
dataarray = dataarray.reshape(len(dataarray), 1)
OHESMILES = onehot_encoder.fit_transform(dataarray).astype(int)
print("Size of one-hot encoded array of data: " + str(OHESMILES.shape))
print("One-hot encoded array of data:")
print(OHESMILES)

Size of one-hot encoded array of data: (25246858, 53)
One-hot encoded array of data:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
#Save OHESMILES as a (compressed) file
np.savez_compressed("ohesmiles.npz", OHESMILES)

In [11]:
#Create integer SMILES data
INTSMILES = [np.where(r==1)[0][0] for r in OHESMILES]

In [12]:
#Save INTSMILES as a (compressed) file
np.savez_compressed("intsmiles.npz", INTSMILES)

In [13]:
#Save array with SMILES character, integer encoding, and one hot encoding (vocabulary)
values = np.reshape(values, (np.shape(values)[0], 1))
vocab = np.concatenate((values, integer_encoded.astype(object)), axis = 1)
vocab = vocab[vocab[:,1].argsort()]
vocabvalues = np.reshape(vocab[:,1], (-1,1))
vocabohe = onehot_encoder.fit_transform(vocabvalues)
vocabencodings = np.concatenate((vocab, vocabohe.astype(object)), axis = 1)
print(np.shape(vocabencodings))

np.save("vocab.npy", vocabencodings)

(53, 55)


In [14]:
print(vocabencodings)

[['#' 0 1.0 ... 0.0 0.0 0.0]
 ['(' 1 0.0 ... 0.0 0.0 0.0]
 [')' 2 0.0 ... 0.0 0.0 0.0]
 ...
 ['r' 50 0.0 ... 1.0 0.0 0.0]
 ['s' 51 0.0 ... 0.0 1.0 0.0]
 ['t' 52 0.0 ... 0.0 0.0 1.0]]
